In [1]:
import streamlit as st
from Bio.Seq import Seq
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Data import CodonTable

def complement(dna_sequence):
    """
    Retorna o complemento reverso da sequência de DNA.

    Argumento:
    - dna_sequence: sequência de DNA no formato 5' 3'

    Retorna:
    - Complemento reverso da sequência de DNA
    """
    seq = Seq(dna_sequence)
    return seq.complement()

def translate(dna_sequence):
    """
    Traduz a sequência de DNA em uma sequência de aminoácidos.

    Argumento:
    - dna_sequence: sequência de DNA no formato 5' 3'

    Retorna:
    - Sequência de aminoácidos traduzida
    """
    seq = Seq(dna_sequence)
    return seq.translate(table=11)  # Usando a tabela de códons padrão (tabela 11)

def find_frames(dna_sequence):
    """
    Encontra as sequências de RNA e aminoácidos para cada frame da sequência de DNA.

    Argumento:
    - dna_sequence: sequência de DNA no formato 5' 3'

    Retorna:
    - Dicionário contendo as sequências de RNA e aminoácidos para cada frame
      no formato:
      {
        frame: {
          'rna': sequência de RNA,
          'aminoacidos': sequência de aminoácidos
        },
        ...
      }
    """
    seq = Seq(dna_sequence)
    frames = {}
    for frame in range(3):
        rna_sequence = seq[frame:].transcribe()
        aminoacid_sequence = rna_sequence.translate(table=11)
        frames[frame + 1] = {
            'rna': str(rna_sequence),
            'aminoacidos': str(aminoacid_sequence)
        }
    return frames

def blast_sequence(dna_sequence):
    """
    Realiza uma pesquisa BLAST para encontrar sequências de nucleotídeos semelhantes.

    Argumento:
    - dna_sequence: sequência de DNA no formato 5' 3'

    Retorna:
    - Lista de resultados BLAST contendo o nome dos nucleotídeos semelhantes e o link para o BLAST correspondente
    """
    result_handle = NCBIWWW.qblast('blastn', 'nt', dna_sequence)
    blast_records = NCBIXML.parse(result_handle)
    blast_results = []
    for blast_record in blast_records:
        for alignment in blast_record.alignments:
            blast_results.append({
                'nome': alignment.title,
                'link': alignment.hit_id
            })
    return blast_results


In [2]:
def main():
    st.title("protKAGG - Tradução e Síntese Protéica")
    st.write("Seu software de tradução e síntese protéica favorito!")

    dna_sequence = st.text_input("Digite a sequência de DNA no formato 5' 3'")
    dna_sequence = dna_sequence.replace(" ", "")

    preview_expander = st.expander("Preview")
    if preview_expander.button("Mostrar preview"):
        complemento = complement(dna_sequence)
        st.write("Complemento Reverso:", complemento)

        aminoacidos = translate(dna_sequence)
        st.write("Aminoácidos:", aminoacidos)

        frames = find_frames(dna_sequence)
        for frame, data in frames.items():
            st.write(f"Frame {frame}:")
            st.write("RNA:", data['rna'])
            st.write("Aminoácidos:", data['aminoacidos'])

        blast_results = blast_sequence(dna_sequence)
        st.write("\nResultados do BLAST:")
        for result in blast_results:
            st.write("Nome:", result['nome'])
            st.write("Link:", result['link'])

if __name__ == '__main__':
    main()

2023-06-06 17:31:00.736 
  command:

    streamlit run c:\venv\ilumpy\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [8]:
"""
GCTCCCGGCTTAGAGGACAGCGGGGAAGGCGGGCGGTGGGGCAGGGGGCCTGAAGCGGCGGTACCGGTGC
TGGCGGCGGCAGCTGAGGCCTTGGCCGAAGCCGCGCGAACCTCAGGGCAAGATGCTTGGAACCGGACCTG
CCGCCGCCACCACCGCTGCCACCACATCTAGCAATGTGAGCGTCCTGCAGCAGTTTGCCAGTGGCCTAAA
GAGCCGGAATGAGGAAACCAGGGCCAAAGCCGCCAAGGAGCTCCAGCACTATGTCACCATGGAACTCCGA
GAGATGAGTCAAGAGGAGTCTACTCGCTTCTATGACCAACTGAACCATCACATTTTTGAATTGGTTTCCA
GCTCAGATGCCAATGAGAGGAAAGGTGGCATCTTGGCCATAGCTAGCCTCATAGGAGTGGAAGGTGGGAA
TGCCACCCGAATTGGCAGATTTGCCAACTATCTTCGGAACCTCCTCCCCTCCAATGACCCAGTTGTCATG
GAAATGGCATCCAAGGCCATTGGCCGTCTTGCCATGGCAGGGGACACTTTTACCGCTGAGTACGTGGAAT
TTGAGGTGAAGCGAGCCCTGGAATGGCTGGGTGCTGACCGCAATGAGGGCCGGAGACATGCAGCTGTCCT
GGTTCTCCGTGAGCTGGCCATCAGCGTCCCTACCTTCTTCTTCCAGCAAGTGCAACCCTTCTTTGACAAC
ATTTTTGTGGCCGTGTGGGACCCCAAACAGGCCATCCGTGAGGGAGCTGTAGCCGCCCTTCGTGCCTGTC
TGATTCTCACAACCCAGCGTGAGCCGAAGGAGATGCAGAAGCCTCAGTGGTACAGGCACACATTTGAAGA
AGCAGAGAAGGGATTTGATGAGACCTTGGCCAAAGAGAAGGGCATGAATCGGGATGATCGGATCCATGGA
GCCTTGTTGATCCTTAACGAGCTGGTCCGAATCAGCAGCATGGAGGGAGAGCGTCTGAGAGAAGAAATGG
AAGAAATCACACAGCAGCAGCTGGTACACGACAAGTACTGCAAAGATCTCATGGGCTTCGGAACAAAACC
TCGTCACATTACCCCCTTCACCAGTTTCCAGGCTGTACAGCCCCAGCAGTCAAATGCCTTGGTGGGGCTG
CTGGGGTACAGCTCTCACCAAGGCCTCATGGGATTTGGGACCTCCCCCAGTCCAGCTAAGTCCACCCTGG
TGGAGAGCCGGTGTTGCAGAGACTTGATGGAGGAGAAATTTGATCAGGTGTGCCAGTGGGTGCTGAAATG
CAGGAATAGCAAGAACTCGCTGATCCAAATGACAATCCTTAATTTGTTGCCCCGCTTGGCTGCATTCCGA
CCTTCTGCCTTCACAGATACCCAGTATCTCCAAGATACCATGAACCATGTCCTAAGCTGTGTCAAGAAGG
AGAAGGAACGTACAGCGGCCTTCCAAGCCCTGGGGCTACTTTCTGTGGCTGTGAGGTCTGAGTTTAAGG"""

'\nGCTCCCGGCTTAGAGGACAGCGGGGAAGGCGGGCGGTGGGGCAGGGGGCCTGAAGCGGCGGTACCGGTGC\nTGGCGGCGGCAGCTGAGGCCTTGGCCGAAGCCGCGCGAACCTCAGGGCAAGATGCTTGGAACCGGACCTG\nCCGCCGCCACCACCGCTGCCACCACATCTAGCAATGTGAGCGTCCTGCAGCAGTTTGCCAGTGGCCTAAA\nGAGCCGGAATGAGGAAACCAGGGCCAAAGCCGCCAAGGAGCTCCAGCACTATGTCACCATGGAACTCCGA\nGAGATGAGTCAAGAGGAGTCTACTCGCTTCTATGACCAACTGAACCATCACATTTTTGAATTGGTTTCCA\nGCTCAGATGCCAATGAGAGGAAAGGTGGCATCTTGGCCATAGCTAGCCTCATAGGAGTGGAAGGTGGGAA\nTGCCACCCGAATTGGCAGATTTGCCAACTATCTTCGGAACCTCCTCCCCTCCAATGACCCAGTTGTCATG\nGAAATGGCATCCAAGGCCATTGGCCGTCTTGCCATGGCAGGGGACACTTTTACCGCTGAGTACGTGGAAT\nTTGAGGTGAAGCGAGCCCTGGAATGGCTGGGTGCTGACCGCAATGAGGGCCGGAGACATGCAGCTGTCCT\nGGTTCTCCGTGAGCTGGCCATCAGCGTCCCTACCTTCTTCTTCCAGCAAGTGCAACCCTTCTTTGACAAC\nATTTTTGTGGCCGTGTGGGACCCCAAACAGGCCATCCGTGAGGGAGCTGTAGCCGCCCTTCGTGCCTGTC\nTGATTCTCACAACCCAGCGTGAGCCGAAGGAGATGCAGAAGCCTCAGTGGTACAGGCACACATTTGAAGA\nAGCAGAGAAGGGATTTGATGAGACCTTGGCCAAAGAGAAGGGCATGAATCGGGATGATCGGATCCATGGA\nGCCTTGTTGATCCTTAACGAGCTGGTCCGAATCAGCAGCATGGAGGGAGAGCGTCTGAGAG